#### colab로 시작하기

<a href="https://colab.research.google.com/github/LDJWJ/00_TOTO_MLDL_CLASS/blob/master/DL04_02_GAN01_BASIC_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GAN(Generative Adversarial Networks)

#### 딥러닝의 미래로 불리는 GAN은 **대립(adversarial)하는 두 신경망을 경쟁**시켜가며 결과물을 생성하는 방법이다.

#### 실제이미지를 구분자(Discriminator)에게 이 이미지가 진짜임을 판단하게 하고, 생성자(Generator)를 통해 노이즈로부터 임의의 이미지를 만들고 이것을 다시 같은 구분자를 통해 진짜 이미지인지를 판단하게 한다.

### 응용 예 <br>

(가) 고흐 풍 그림으로 그려주기 <br>
(나) 선으로 그려진 만화를 자동으로 채색 <br>
(다) 모자이크를 없애주기 <br>
(라) GAN 기법을 이용한 자연어 문장 생성

### 이번시간의 학습 내용
* 손글씨 데이터 셋(MNIST)을 이용하여 손글씨를 무작위로 생성하는 예제 만들기

In [1]:
# 논문링크 : https://arxiv.org/abs/1406.2661

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as ml

print("tf : {}, numpy : {}, matplotlib : {}".format(tf.__version__, 
                                                   ml.__version__, np.__version__))

tf : 1.15.0, numpy : 3.2.0, matplotlib : 1.18.2


In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
print(mnist.test.images.shape)
print(mnist.test.labels.shape)
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(10000, 784)
(10000, 10)
(55000, 784)
(55000, 10)


In [4]:
print(type(mnist.train.images) )

<class 'numpy.ndarray'>


In [5]:
print(mnist.train.labels[0:10])

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


### 01. 기본 옵션 설정

In [0]:
total_epoch = 100       # epoch 수 설정
batch_size = 100        # 배치 사이즈
learning_rate = 0.0002  # 학습률

# 신경망 레이어 구성 옵션
n_hidden = 256          # 은닉층 노드
n_input = 28 * 28       # 입력 
n_noise = 128           # 생성기의 입력값으로 사용할 노이즈의 크기

### 02. 신경망 모델 구성
 * **노이즈**를 이용하여 데이터 생성
 * 비지도학습이므로 목표(Y)가 없음

In [0]:
# GAN 도 Unsupervised 학습이므로 Autoencoder 처럼 Y 를 사용하지 않습니다.
X = tf.placeholder(tf.float32, [None, n_input])

# 노이즈 Z를 입력값으로 사용합니다.
Z = tf.placeholder(tf.float32, [None, n_noise])

### 생성자 신경망, 판별자 신경망 변수 선언

In [8]:
# 생성자 신경망에 사용하는 변수(Weight, bias)
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))

G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

print(G_W1, G_W2)
print(G_b1, G_b2)

<tf.Variable 'Variable:0' shape=(128, 256) dtype=float32_ref> <tf.Variable 'Variable_2:0' shape=(256, 784) dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=(256,) dtype=float32_ref> <tf.Variable 'Variable_3:0' shape=(784,) dtype=float32_ref>


In [9]:
# 판별기 신경망에 사용하는 변수들입니다.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

# 판별기의 최종 결과값은 얼마나 진짜와 가깝냐를 판단하는 한 개의 스칼라값입니다.
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

print(D_W1, D_W2)
print(D_b1, D_b2)

<tf.Variable 'Variable_4:0' shape=(784, 256) dtype=float32_ref> <tf.Variable 'Variable_6:0' shape=(256, 1) dtype=float32_ref>
<tf.Variable 'Variable_5:0' shape=(256,) dtype=float32_ref> <tf.Variable 'Variable_7:0' shape=(1,) dtype=float32_ref>


### 2-1 생성자(G) 신경망 구성
 * 무작위 생성한 노이즈를 받아, 가중치와 편향을 반영하여 은닉층 구성.
 * sigmoid 함수를 이용하여 최종 결과값 0~1 사이의 값 반환

In [0]:
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, G_W2) + G_b2)

    return output

### 2-2 구분자(D) 신경망 구성
 * 구분자 신경망 구성, 가중치와 편향을 반영한 데이터 출력
 * sigmoid 함수를 이용하여 최종 결과값 0~1 사이의 값 반환

In [0]:
def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, D_W2) + D_b2)

    return output

### 2-3 생성자 신경망의 노이즈 발생을 위한 노이즈 생성 함수

In [0]:
# 랜덤한 노이즈(Z)를 만듭니다.
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

### 2-4 신경망 구성

In [13]:
# 노이즈를 이용해 랜덤한 이미지를 생성합니다.
# Z에는 실행 시, noise가 입력됨.
G = generator(Z)
print("generator :", G)

# 판별자 신경망(구분자) 구성 - 생성된 이미지가 들어갈때
# 노이즈를 이용해 생성한 이미지(G)가 진짜 이미지인지 판별한 값을 구합니다.
D_fake = discriminator(G)
print("discriminator use Noise Image:", D_fake)

# 판별자(구분자) 신경망 구성 - 실제 이미지 데이터
# 진짜 이미지를 이용해(X) 판별한 값을 구합니다.
D_real = discriminator(X)
print("discriminator use Real Image:", D_real)

generator : Tensor("Sigmoid:0", shape=(?, 784), dtype=float32)
discriminator use Noise Image: Tensor("Sigmoid_1:0", shape=(?, 1), dtype=float32)
discriminator use Real Image: Tensor("Sigmoid_2:0", shape=(?, 1), dtype=float32)


 * GAN은 생성자(Generator) : **구분자가 1로 예측하도록 하는 것을 목표**로 학습시킴. 
 * GAN은 구분자(Discriminator) : **진짜 데이터를 받으면 1**로 **가짜 데이터를 받으면 0으로 예측**하도록 학습시킴.

### GAN의 모델의 손실함수와 최적화 함수
 * 두개의 손실값이 필요
   * 위조 지폐 판단 - D_fake (생성된 이미지에 대한 신경망)
   * 진품 판단 - D_real(실제 이미지에 대한 신경망)

### 경찰을 학습시키려면 
  * D_fake(가짜판단)은 최대한 0에 가깝게, D_real(진품 판단)은 최대한 1에 가깝게

 * 수식 표현
   * 신경망에서는 보통 비용함수, 손실함수라고 한다. 하지만 GAN에서는 이 손실함수 또는 비용함수를 극대화하므로 개인적으로 이를 **trust로 하여 신뢰도**라고 표기를 하겠음.
   * trust_D = tf.log(진품판단값) + tf.log(1 - 가짜판단값)의 평균
   * trust_D는 D_real이 1에 가까워지고, D_fake가 0에 가까워지면 값은 커지게 된다.


In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
a = sess.run(tf.log(1.0))   # log는 1에 가까워질수록 0에 가까워지고,
b = sess.run(tf.log(0.0))   # log가 0에 가까워질수록 -무한대로 가게 된다.
c = sess.run(tf.log(0.1))   
sess.close()
print(a,b,c)

0.0 -inf -2.3025851


In [0]:
# 가장 큰 값을 갖게 되는 것은 D_real가 1에 가까워지고, 1-D_fake가 1에 가까워질때.
# trust_D가 크도록 학습시켜 가야 함. (판별기 신경망의 신뢰도-비용함수)
# 진짜 이미지 넣을 때는 크도록 값을 하고, 가짜 이미지를 넣을 때는 반대의 경우가 커야 하므로 아래와 같이 지정하여 계산)
trust_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_fake))

### 위조 지폐범 학습
   * D_fake(가짜판단)를 최대한 1에 가깝게 하기
   * 수식 표현 : trust_G = tf.log(D_fake)의 평균값

In [0]:
# 결국 D_fake 값을 1에 가깝게 새로운 이미지를 생성하는 것이 위조지폐범의 목표이므로,
# trust_G를 큰 값을 가지도록 학습시키는 것이 목표임.
# 생성망 신경망의 비용함수(생성망 신경망의 신뢰도)는
# 가짜 이미지 넣을 때의 신경망의 값이 최대한 1에 가깝게 해야 하므로 아래와 같이 지정하여 계산)
trust_G = tf.reduce_mean(tf.log(D_fake))

### GAN의 학습은 최종적으로
  * trust_D (경찰)의 학습을 통해 값을 최대화 (값을 크게)
  * trust_G (위조지폐범)의 학습을 통해 값을 최대화 (값을 크게)

* 각각의 신경망의 비용함수(손실함수)인 trust_G와 trust_D를 최대화 하는 것. 
   * 단, 서로의 손실이 연관되어 있어, 두 손실값이 같이 증가가 어려움.

In [0]:
# trust_D 를 구할 때는 판별기 신경망에 사용되는 변수만 사용하고,
# trust_G 를 구할 때는 생성기 신경망에 사용되는 변수만 사용하여 최적화를 합니다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [18]:
# GAN 논문의 수식에 따르면 비용함수를 극대화 해야하지만, 
# 텐서플로워에서 minimize 하는 최적화 함수를 사용하기 때문에
# 최적화 하려는 trust_D 와 trust_G 에 음수 부호를 붙여줍니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-trust_D,
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-trust_G,
                                                         var_list=G_var_list)

print(train_D)
print(train_G)

name: "Adam"
op: "NoOp"
input: "^Adam/update_Variable_4/ApplyAdam"
input: "^Adam/update_Variable_5/ApplyAdam"
input: "^Adam/update_Variable_6/ApplyAdam"
input: "^Adam/update_Variable_7/ApplyAdam"
input: "^Adam/Assign"
input: "^Adam/Assign_1"

name: "Adam_1"
op: "NoOp"
input: "^Adam_1/update_Variable/ApplyAdam"
input: "^Adam_1/update_Variable_1/ApplyAdam"
input: "^Adam_1/update_Variable_2/ApplyAdam"
input: "^Adam_1/update_Variable_3/ApplyAdam"
input: "^Adam_1/Assign"
input: "^Adam_1/Assign_1"



### 03. 모델 학습

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
trust_val_D, trust_val_G = 0, 0

* 학습 후, 학습되는 이미지 저장을 위해 실행되는 위치에 '/samples'라는 디렉터리(폴더) 생성이 필요함.

In [20]:
%%time

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        # 판별기와 생성기 신경망을 각각 학습시킵니다.
        _, trust_val_D = sess.run([train_D, trust_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, trust_val_G = sess.run([train_G, trust_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          '경찰(D) 신뢰도(trust): {:.4}'.format(trust_val_D),
          '위조지폐범(G) 신뢰도(trust): {:.4}'.format(trust_val_G))
                                 
    #########
    # 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
    ######
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 경찰(D) 신뢰도(trust): -0.4006 위조지폐범(G) 신뢰도(trust): -2.243
Epoch: 0001 경찰(D) 신뢰도(trust): -0.3384 위조지폐범(G) 신뢰도(trust): -2.434
Epoch: 0002 경찰(D) 신뢰도(trust): -0.1236 위조지폐범(G) 신뢰도(trust): -3.11
Epoch: 0003 경찰(D) 신뢰도(trust): -0.678 위조지폐범(G) 신뢰도(trust): -1.552
Epoch: 0004 경찰(D) 신뢰도(trust): -0.3229 위조지폐범(G) 신뢰도(trust): -2.156
Epoch: 0005 경찰(D) 신뢰도(trust): -0.4149 위조지폐범(G) 신뢰도(trust): -2.385
Epoch: 0006 경찰(D) 신뢰도(trust): -0.2632 위조지폐범(G) 신뢰도(trust): -2.746
Epoch: 0007 경찰(D) 신뢰도(trust): -0.2964 위조지폐범(G) 신뢰도(trust): -2.527
Epoch: 0008 경찰(D) 신뢰도(trust): -0.3429 위조지폐범(G) 신뢰도(trust): -2.495
Epoch: 0009 경찰(D) 신뢰도(trust): -0.2209 위조지폐범(G) 신뢰도(trust): -2.797
Epoch: 0010 경찰(D) 신뢰도(trust): -0.3182 위조지폐범(G) 신뢰도(trust): -2.411
Epoch: 0011 경찰(D) 신뢰도(trust): -0.2695 위조지폐범(G) 신뢰도(trust): -2.899
Epoch: 0012 경찰(D) 신뢰도(trust): -0.2217 위조지폐범(G) 신뢰도(trust): -2.805
Epoch: 0013 경찰(D) 신뢰도(trust): -0.2691 위조지폐범(G) 신뢰도(trust): -3.188
Epoch: 0014 경찰(D) 신뢰도(trust): -0.4097 위조지폐범(G) 신뢰도(trust): -2.341
Epoch: 0015 

### REF 
* GAN에 대한 논문 : https://arxiv.org/abs/1406.2661

Copyright 2020 LIM Co.(예영Edu Co.) all rights reserved. <br>
교육용으로 작성된 것으로 배포 및 복제시에 사전 허가가 필요합니다. <br>